In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


In [37]:
# import training data
dft_eng = pd.read_csv('../data/dft_eng.csv')
dft_jap = pd.read_csv('../data/dft_jap.csv')
dft_fin = pd.read_csv('../data/dft_fin.csv')

# import validation data
dfv_eng = pd.read_csv('../data/dfv_eng.csv')
dfv_jap = pd.read_csv('../data/dfv_jap.csv')
dfv_fin = pd.read_csv('../data/dfv_fin.csv')

In [38]:
dft_eng

,question_text,document_title,language,annotations,document_plaintext,document_url,answer_start,answer_text,question_text_tokenized,document_plaintext_tokenized,answer_text_tokenized,labels
0,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': array([159]), 'answer_text': ...",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,[159],['1920s'],"['When', 'was', 'quantum', 'field', 'theory', ...","['Quantum', 'field', 'theory', 'naturally', 'b...",['1920s'],1
1,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': array([610]), 'answer_text': ...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,[610],['Sully Prudhomme'],"['Who', 'was', 'the', 'first', 'Nobel', 'prize...","['The', 'Nobel', 'Prize', 'in', 'Literature', ...","['Sully', 'Prudhomme']",1
2,When is the dialectical method used?,Dialectic,english,"{'answer_start': array([129]), 'answer_text': ...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,[129],['discourse between two or more people holding...,"['When', 'is', 'the', 'dialectical', 'method',...","['Dialectic', 'or', 'dialectics', '(', 'Greek'...","['discourse', 'between', 'two', 'or', 'more', ...",1
3,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': array([88]), 'answer_text': a...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,[88],['Sejong the Great'],"['Who', 'invented', 'Hangul', '?']","['Hangul', 'was', 'personally', 'created', 'an...","['Sejong', 'the', 'Great']",1
4,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': array([0]), 'answer_text': ar...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,[0],"['Grasshoppers are plant-eaters, with a few sp...","['What', 'do', 'Grasshoppers', 'eat', '?']","['Grasshoppers', 'are', 'plant-eaters', ',', '...","['Grasshoppers', 'are', 'plant-eaters', ',', '...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
7384,What was Neil Brooks' fastest recorded time?,Swimming at the 1980 Summer Olympics – Men's 4...,english,"{'answer_start': array([-1]), 'answer_text': a...",The medley relay was scheduled in the Olympisk...,https://en.wikipedia.org/wiki/Swimming%20at%20...,[-1],[''],"['What', 'was', 'Neil', 'Brooks', ""'"", 'fastes...","['The', 'medley', 'relay', 'was', 'scheduled',...",[],0
7385,Who are the three most important eastern philo...,Eastern philosophy,english,"{'answer_start': array([-1]), 'answer_text': a...",Sāmkhya is a dualist philosophical tradition b...,https://en.wikipedia.org/wiki/Eastern%20philos...,[-1],[''],"['Who', 'are', 'the', 'three', 'most', 'import...","['Sāmkhya', 'is', 'a', 'dualist', 'philosophic...",[],0
7386,Who was costume designer for the first Star Wa...,John Mollo,english,"{'answer_start': array([-1]), 'answer_text': a...",Mollo was surprised by the success of Star War...,https://en.wikipedia.org/wiki/John%20Mollo,[-1],[''],"['Who', 'was', 'costume', 'designer', 'for', '...","['Mollo', 'was', 'surprised', 'by', 'the', 'su...",[],0
7387,Who developed the first thermonuclear weapon?,History of nuclear weapons,english,"{'answer_start': array([-1]), 'answer_text': a...","In the end, President Truman made the final de...",https://en.wikipedia.org/wiki/History%20of%20n...,[-1],[''],"['Who', 'developed', 'the', 'first', 'thermonu...","['In', 'the', 'end', ',', 'President', 'Truman...",[],0


In [77]:
def ans_freq(que, doc):

    freq = [x for x in que if x in doc]
    freq = len(freq)/len(que)
    return freq

0.42857142857142855

In [86]:
# calculating frequenzy of words in answer that are in text 
frequency = []
for question, answer in zip(dft_eng['question_text_tokenized'], dft_eng['document_plaintext_tokenized']):
    frequency.append(ans_freq(question, answer))

print((np.array(frequency).shape))

(7389,)
